# Game Builder

## Setup

### Install Dependencies

In [1]:
%pip install nltk
%pip install langchain
%pip install openai==1.54.0
%pip install httpx==0.27.2
#%pip install marisa-trie
#%pip install graphviz
#%pip install transformers
#%pip install scikit-learn


# # For calculating cosine similarity
# %pip install chromadb
# %pip install tiktoken

# For safe eval
%pip install simpleeval

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Import libraries

In [2]:
%load_ext autoreload
%autoreload 2
from langchain.schema.language_model import BaseLanguageModel
# from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
# from langchain.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate
# from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain import LLMChain, PromptTemplate
from typing import *
import sys, json, os, random, re, math
from copy import deepcopy
from utils import *
import nltk # type: ignore
nltk.download('wordnet') # type: ignore


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ericzhou/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Load LLM (Choose 1 only)

#### GPT-3.5 or 4o-mini

In [ ]:
###MUST PROVIDE YOUR OWN API_KEY in llm/openai_key.txt
from llm.chatgpt import ChatGPT
llm = ChatGPT("gpt-4o-mini", temperature = 0)

#### Test LLM Response

In [4]:
###MUST PROVIDE YOUR OWN API_KEY in llm/openai_key.txt
llm.get_response("Hello, how are you?")

"Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?"

### Initialize Generated Story

In [10]:
with open("result/Disarmed_Bomb.json", "r") as f:
    game_input = json.load(f)
print(game_input)

{'game_logics': {'sneak past the outer guards': {'item needed': [], 'location': ['at facility perimeter'], 'preceding_events': [], 'description': [], 'results': ['at control room']}, 'hack into the surveillance terminal': {'item needed': [], 'location': ['in control room'], 'preceding_events': ['sneak past the outer guards'], 'description': [], 'results': []}, 'extract the bomb access code': {'item needed': [], 'location': ['in server room'], 'preceding_events': ['hack into the surveillance terminal'], 'description': [], 'results': ['has bomb access code']}, 'neutralize the bomb technician': {'item needed': [], 'location': ["in technician's quarters"], 'preceding_events': ['extract the bomb access code'], 'description': [], 'results': []}, 'disarm the bomb': {'item needed': ['has bomb access code'], 'location': ['at bomb site'], 'preceding_events': ['neutralize the bomb technician'], 'description': [], 'results': []}}, 'map': {'at facility perimeter': [{'type': 'npc', 'content': 'outer

#### Some necessary functions for us to generate a playable game.

In [ ]:
from game_construct_prompt import expand_sentence, analyze_action, populate_attribute, get_verbs, generate_new_preconditions, check_if_existing_action, check_future_events
from game import Game
from world import World
from nodes import Node, Item, Room, Character, Player, ContainerItem
from logic_template import ActionTemplate, EventTemplate
from condition import ComplexCondition
from type import Coordinate
import numpy as np
import traceback


class GameBuilderHelper:

    @staticmethod
    def get_topological_ordering_of_game_logics(game_input: Dict[str, Any]) -> Dict[str, Any]:
        """
        Returns a topological ordering of the game logics. Events that must happen before other events are ordered first.

        Args:
            game_input (Dict[str, Any]): Dictionary containing game's logic definitions.

        Returns:
            Dict[str, Any]: Dictionary containing game's logic definitions with events ordered topologically.
        """
        game_logics = game_input['game_logics']

        # Define the possible states for nodes during DFS
        UNVISITED = 0
        VISITING = 1
        VISITED = 2

        # Initialization
        order: List[str] = []  # Stores the topological order of nodes (events)
        # Track state of each node
        state = {event: UNVISITED for event in game_logics}

        #print("STATE")
        #print(state)

        def dfs(event: str) -> bool:
            """Recursive helper function to perform DFS and check for cycles."""
            if state[event] == VISITING:
                # Cycle detected, topological order is not possible
                return False
            if state[event] == VISITED:
                # Already processed, skip
                return True

            # Mark current node as VISITING
            state[event] = VISITING

            # Visit all neighbors
            for preceding_event in game_logics[event]['preceding_events']:
                if not dfs(preceding_event):
                    return False

            # Mark current node as VISITED and add to order
            state[event] = VISITED
            order.append(event)

            return True

        # For each node, if it's unvisited, perform DFS
        for event in game_logics:
            if state[event] == UNVISITED and not dfs(event):
                # If there's a cycle, return an empty list or raise an error
                raise ValueError(
                    "Cycle detected, topological order is not possible.")
        
        # Return the game logics with events ordered topologically
        return {event: game_input['game_logics'][event] for event in order}

    @staticmethod
    def _expand_sentences(game_input: Dict[str, Any]) -> Tuple[Dict[str, str], Dict[str, List[str]]]:
        """
        Expands game logic sentences and extracts objects involved in them.

        For each sentence in game_input['game_logics'], it expands the sentence and identifies the objects 
        associated with the sentence. For example, "Adventurer crafts sword" will be expanded to "Adventurer crafts sword with iron and anvil."

        Args:
            game_input (Dict[str, Any]): Dictionary containing game's logic definitions.

        Returns:
            Tuple[Dict[str, str], Dict[str, List[str]]]: 
            - Dictionary mapping original sentences to their expanded form.
            - Dictionary mapping original sentences to objects involved in them.

        """
        expanded_sentences: Dict[str, str] = {}
        objects_in_expanded_sentences: Dict[str, List[str]] = {}
        for sentence in game_input['game_logics']:
            expanded_sentence, objects = (expand_sentence(
                llm, sentence, game_input['game_logics'][sentence]['item needed']))
            expanded_sentences[sentence] = expanded_sentence
            objects_in_expanded_sentences[sentence] = objects

        return expanded_sentences, objects_in_expanded_sentences

    @staticmethod
    def expand_game_logics(game_input: Dict[str, Any]) -> Dict[str, Any]:
        """
        Updates game logic with expanded sentences.

        For each game logic sentence, the function replaces original sentences with their expanded form 
        and updates any preceding events.

        Args:
            game_input (Dict[str, Any]): Original game logic definitions.
            expanded_sentences (Dict[str, str]): Dictionary mapping original sentences to their expanded form.

        Returns:
            Dict[str, Any]: Updated game logics.
        """
        expanded_sentences, _ = GameBuilderHelper._expand_sentences(game_input)
        new_game_logics = deepcopy({expanded_sentence: game_input['game_logics'][original_sentence]
                                   for original_sentence, expanded_sentence in expanded_sentences.items()})
        for sentence in new_game_logics:
            old_preceding_events: List[str] = new_game_logics[sentence]['preceding_events']
            new_preceding_events: List[str] = []
            for old_preceding_event in old_preceding_events:
                
                if old_preceding_event in expanded_sentences:
                    new_preceding_events.append(
                        expanded_sentences[old_preceding_event])
                else:
                    print_warning(
                        f'Warning: {old_preceding_event} not in expanded_sentences!', 'update_game_with_expanded_sentences')
                    new_preceding_events.append(old_preceding_event)
            new_game_logics[sentence]['preceding_events'] = new_preceding_events

            #print("COMPARING EXPANDED LOGICS")
            #print(sentence)
            #print(old_preceding_events)
            #print(new_preceding_events)
        return new_game_logics
    
    @staticmethod
    def add_nodes_from_game_logics(world: World, sentence: str, game_logics_content: Dict[str, Any], parsed_arguments_in_sentence: Dict[str, Union[str, List[str]]]) -> Room:

        # 1: Get the room where the action takes place. 
        # If it doesn't exist, add it to the world (this should not happen often).
        location_name = remove_extra_spaces(
            game_logics_content['location'][0])
        room = GameBuilderHelper.add_node_if_not_exist(world, location_name, Room,
                                                       (0, world.get_num_rooms()))
        assert isinstance(room, Room)
        # 2: Get the items in "item needed".
        # If they don't exist, add them to the room.

        for item_name in game_logics_content['item needed']:
            #print("ITEM_NAME", item_name)
            GameBuilderHelper.add_node_if_not_exist(
                world, item_name, Item, room,
                #not_exist_callback=lambda: print_warning(
                #    f'Warning: {item_name} appears in game logics but not in map! Adding {item_name} to world', 'get_world')
            )           
        # 3: Get the characters and items involved in the sentence. 
        # If they don't exist, add them to the room.

        for placeholder in parsed_arguments_in_sentence:
            node_names = parsed_arguments_in_sentence[placeholder]
            nodes: List[Node] = []

            if isinstance(node_names, str):
                node_names = [node_names]
            if 'item' in placeholder:
                for node_name in node_names:
                    node = GameBuilderHelper.add_node_if_not_exist(
                        world, node_name, Item, room,
                        #not_exist_callback=lambda: print_warning(
                        #    f'Warning: {node_name} appears in game logics but not in map! Adding {node_name} to world', 'get_world')
                    )
                    nodes.append(node)
            elif 'container' in placeholder:
                for node_name in node_names:
                    node = GameBuilderHelper.add_node_if_not_exist(
                        world, node_name, ContainerItem, room,
                        #not_exist_callback=lambda: print_warning(
                        #    f'Warning: {node_name} appears in game logics but not in map! Adding {node_name} to world', 'get_world')
                    )
                    nodes.append(node)
            elif 'character' in placeholder:
                for node_name in node_names:
                    node = GameBuilderHelper.add_node_if_not_exist(
                        world, node_name, Character, room,
                        #not_exist_callback=lambda: print_warning(
                        #    f'Warning: {node_name} appears in game logics but not in map! Adding {node_name} to world', 'get_world')
                    )
                    nodes.append(node)
            elif 'room' in placeholder:
                for room_name in node_names:
                    #if not world.node_exists(room_name):
                        #print_warning(
                        #    f'Warning: {room_name} is a room that does not appear in the map! Ignoring', 'get_world')
                    #else:
                    nodes.append(world.find_node(room_name))
            else:
                print_warning(
                    f'Warning: {parsed_arguments_in_sentence[placeholder]} is not an item or character! Ignoring', 'get_world')
            
            # TODO: Populate the game with hidden items and characters that are not explicitly mentioned in the game logics or map.
            # This makes generation too slow, so removed from this version.
        return room

    @staticmethod
    def add_node_if_not_exist(world: World, name: str, type: Type[Node], container: Union[Coordinate, Node], description: str = '', exist_callback: Union[None, Callable[[], None]] = None, not_exist_callback: Union[None, Callable[[], None]] = None, node_added_callback: Union[None, Callable[[], None]] = None, **kwargs: Any) -> Node:
        #print("ADDING NODE IF NOT EXIST")
        name = remove_extra_spaces(name)
        if type == Room and name.lower().startswith('at '):
            name = name[3:]
        elif type in [Item, Character, ContainerItem] and name.lower().startswith('has '):
            name = name[4:]
        #name = get_lemma(name)
        if not description:
            # TODO: Better description
            if type == Room:
                description = 'You are at ' + name + '.'
            else:
                description = 'An ' + \
                    name if name[0].lower() in ['a', 'e', 'i', 'o',
                                                'u'] else 'A ' + name + '.'
        try:
            if type == Room:
                node = world.find_node(name, strict=True)
            else:
                assert isinstance(container, Node)
                try:
                    node = world.find_node(
                        name, room=container.get_room().id, local=True, strict=True)
                except:
                    node = world.find_node(name, strict=True)
            # node exists.
            if exist_callback:
                exist_callback()
            return node
        except:
            pass
        if not_exist_callback:
            not_exist_callback()
        node = type(name, description=description)
        for attribute_name, attribute_value in kwargs.items():
            node.set_attribute(attribute_name, attribute_value)
        #print("ADDING NODE")
        world.add_node(node, container)
        if node_added_callback:
            node_added_callback()
        return node

class GameBuilder:
    def __init__(self, game_input: Dict[str, Any]) -> None:
        self.game_input = game_input
        world = World(configurations={"map_size": (1, 128)})
        self.game = Game(world, [], [])
        self.replay_games: List[Game] = [deepcopy(self.game)]

    def preprocess_game_input(self) -> None:
        game_input = self.game_input
        game_logics = game_input['game_logics']

        # Step 1: Get a topological ordering of game logics: Events that must happen before other events are ordered first.
        game_logics = GameBuilderHelper.get_topological_ordering_of_game_logics(
            game_input)
        
        #print("DONE ORDERING")
        #print(game_logics)

        # Step 2: Expand game logics: Add objects to sentences and expand them
        game_logics = GameBuilderHelper.expand_game_logics(game_input)

        #print("DONE EXPANDING")
        #print(game_logics)

        game_input['game_logics'] = game_logics

        self.game_input = game_input

    def initialize_world(self) -> None:
        """
        Generates a game world based on game input.

        Only rooms, characters, and items defined in game_input['map'] are added to the world.

        Args:
            game_input (Dict[str, Any]): Dictionary containing game's configurations and logic definitions.

        Returns:
            World: The populated game world.

        """
        game_input = self.game_input

        world = self.game.world

        # Add rooms, characters, and items defined in game_input['map']
        #print("ADDING NODES HERE")
        for room_name in game_input['map']:
            pattern = r"\b(?:at|has)\b\s*"
            cleaned_room_name = re.sub(pattern, '', room_name).strip()
            room = GameBuilderHelper.add_node_if_not_exist(world, cleaned_room_name, Room, (
                0, world.get_num_rooms()), node_added_callback=lambda: log('get_world', f'Adding Room {cleaned_room_name} to world'))
            for node in game_input['map'][room_name]:
                node_name = node['content']
                cleaned_node_name = re.sub(pattern, '', node_name).strip()
                if node['type'] == 'npc':
                    GameBuilderHelper.add_node_if_not_exist(world, cleaned_node_name, Character, room, node_added_callback=lambda: log(
                        'get_world', f'Adding Character {cleaned_node_name} to world'))
                elif node['type'] == 'item':
                    GameBuilderHelper.add_node_if_not_exist(world, cleaned_node_name, Item, room, node_added_callback=lambda: log(
                        'get_world', f'Adding Item {cleaned_node_name} to world'))

        # TODO: Player goal and description should be provided or generated
        first_room = world.get_room_from_coordinate((0, 0))
        assert first_room is not None
        player = GameBuilderHelper.add_node_if_not_exist(world, 'player', Player, first_room, node_added_callback=lambda: log(
            'get_world', f'Adding Player to world'), goal="Your goal is resolve the curse of the werewolf")
        # TODO: This is a hack to make sure there is money in the game. Should be removed.
        #GameBuilderHelper.add_node_if_not_exist(world, 'money', Item, player)

    def _check_action_effects(self, sentence: str) -> Tuple[bool, str]:
        """
        Checks if an action has the correct effects.

        The function checks if the action has the correct effects. If it does, it returns True. Otherwise, it returns False and a message explaining why the action is invalid.

        Args:
            sentence (str): The action to be checked.

        Returns:
            Tuple[bool, str]: 
            - True if the action has the correct effects, False otherwise.
            - A message explaining why the action is invalid if it's not added to the game.
        """
        raise NotImplementedError('Not implemented yet')
    
    def add_dynamic_action(self, user_input: str) -> Tuple[bool, str, str]:
        items, attributes, complete_sentences, output_list = generate_new_preconditions(self.game, llm, user_input, [])
        #print("NEW ACTION OUTPUT")
        #print(output_list)
        valid_rooms = []
        for node in adventureGame.world.nodes:
            if type(node) is Room:
                valid_rooms.append(node)
        for item in items:
            rand_room = np.random.choice(valid_rooms)
            GameBuilderHelper.add_node_if_not_exist(adventureGame.world, item, Item, rand_room, node_added_callback=lambda: log(
                        'get_world', f'Adding Item {item} to world'))

    def _try_adding_action(self, sentence: str, previous_attempts:List[Tuple[str, str]]=[]) -> Tuple[bool, str, str, ActionTemplate]:
        """
        Tries to add an action to the game.

        The function tries to add an action to the game. If the action is valid, it adds it to the game and returns True.
        Otherwise, it returns False and a message explaining why the action is invalid.

        Args:
            sentence (str): The action to be added to the game.

        Returns:
            Tuple[bool, str, str]: 
            - True if the action is valid and added to the game, False otherwise.
            - LLM Raw Output
            - A message explaining why the action is invalid if it's not added to the game.
        """
        assert sentence in self.game_input['game_logics'], f'{sentence} not in game_input["game_logics"]'
        llm_raw_output = f'Error! Failed to analyze action: {sentence}'
        #print("START")
        #print(self.game_input['game_logics'][sentence]['location'])
        preceding_events = self.game_input['game_logics'][sentence]['preceding_events']
        input = sentence + "; room: " + self.game_input['game_logics'][sentence]['location'][0] + "."
        try:
            #print("Right before analyze")
            #print(analyze_action.__code__.co_varnames)
            #print(analyze_action.__code__.co_argcount)              
            llm_raw_output, base_form, arguments, attribute_list, action_template = analyze_action(self.game,
                llm, input, previous_attempts)  # Example: {"character": ["adventurer"], "item": ["money"]}
            #log('_try_adding_action', f'\n{llm_raw_output}')
            #print("ACTION TEMPLATE")
            #print(action_template)
            world = self.game.world
            #print("sub 1")

            # STEP 1: Register attributes.
            for attribute_name, belonging_class_name, attribute_type in attribute_list:
                belonging_class = Node.mapping[belonging_class_name]
                if not belonging_class.has_attribute(attribute_name):
                    belonging_class.register_new_attribute(
                        attribute_name, attribute_type, None)
            #print(attribute_list)
            #print("sub 2")
            #print("SENTENCE")
            #print(sentence)

            # STEP 2: Identify nodes involved in the action. 
            # Add mising characters and items in game_input['game_logics']. 
            # This should not happen often.
            
            room = GameBuilderHelper.add_nodes_from_game_logics(world, sentence, self.game_input['game_logics'][sentence], arguments)
            
            #print("sub 3")

            # STEP 3: Move player to the room where the action takes place, assuming that the player can go there.
            # TODO: If the player can't go to the room, the action is invalid.
            player = world.player
            assert player is not None, 'Player is not in the world!'
            if player.get_room() != room:
                world.move_node(player, room)
            #print("SENTENCE")
            #print(sentence)

            # STEP 4: Add action, and make sure the preconditions are met
            # Populate node attributes if they are not specified. objects in the sentence are assumed to satisfy the conditions in game logics.
            # In condition, if attribute is not specified (having a value of None), make it satisfy the condition.
            # check if the condition is a simple condition consisting of only the "and" operator.
            # If it is simple, edit node attributes to satisfy the condition.
            # If it is complex, prompt llm to edit node attributes to satisfy the condition.
            # If it still fails, return failure.

            #print("sub 4")
            #print(action_template)
            self.game.add_action_template(action_template)
            #print("BASEFORM")
            #print(base_form)
            command = replace_placeholders(base_form, arguments)
            #print(command)
            action = self.game.get_action(command)
            action.input_name = sentence
            for flag in self.game_input['game_logics'][sentence]['preceding_events']:
                action.flags.append(flag)
            assert action is not None, "Bug! Action is none!"
            #print("GETTING FIXES")
            fixes = action.conditions.get_fixes_complex(self.game, llm=llm)
            for fix in fixes:
                fix.apply(self.game.world)
            
            #add_verbs = get_verbs(base_form)
            ###How do we identify which verbs have corresponding attributes that we care about? i.e. we want to add "throw" but what about "get" or its synonyms, like "obtain"
            #for node_name in add_verbs:
            #    attributes = add_verbs[node_name]
            #    node = world.find_node(node_name)
            #    for attribute in attributes:
            #        world.register_new_attribute(Node, attribute)
            #        ##More work here to understand what register_new_attribute is actually doing here.
            #        node.register_new_attribute(None, attribute, None)
            
            # STEP 5: Apply the action. The action may fail.
            #print("EXECUTING ACTIONS")
            #print(action)
            action_result = action.execute(self.game)
            self.game.happened_events.add(sentence)
            #print(action_result)

            # STEP 6: Check if the action succeeds.
            #print(self.game.action_history)
            action_result = self.game.action_history[-1]
            if not action_result.success:
                # REMOVE THE ACTION IF ALREADY ADDED
                #print("REMOVE NOT WORKING ACTION", self.game.remove_action(action_template))
                return False, llm_raw_output, action_result.observation, None

            # STEP 7: Check if the action has the correct effects.
            desired_effects: List[str] = self.game_input['game_logics'][sentence]['results']
            desired_effects = [f'{{{effect}}}' for effect in desired_effects]
            actual_effects_expression = ' and '.join(desired_effects)
            #print(actual_effects_expression)
            actual_effects_condition = ComplexCondition.build_from_string(
                game = self.game, expression = actual_effects_expression)
            is_satisfied, messages = actual_effects_condition.evaluate(
                self.game)
            #print("ACTUAL EFFECTS SATISFACTION CHECK")
            #print(is_satisfied, messages)

            #if not is_satisfied:
            #    return False, llm_raw_output, '\n'.join(messages)
            
        except Exception as e:
            log('_try_adding_action', f"Caught exception: {e}")
            traceback.print_exc()
            #print("REMOVE NOT WORKING ACTION", self.game.remove_action(action_template))
            return False, llm_raw_output, str(e), None

        # Add command to list of ideal action
        self.game.commands.append(command)
        return True, llm_raw_output, '', action_template

    def save_game_snapshot(self) -> None:
        self.replay_games.append(deepcopy(self.game))

    def populate_node_attributes(self) -> None:
        nodes = self.game.world.nodes
        for node in nodes:
            _, attribute_values = populate_attribute(llm, node)
            for attribute, value in attribute_values.items():
                #print("POPULATING AND SETTING NODE ATTRIBUTES")
                #print(node, attribute, value)
                node.set_attribute(attribute, value)

    # NOT TESTED!!!
    def build_game(self) -> Game:


        # Step 1: Preprocess game input
        #print("1")
        self.preprocess_game_input()

        #print("NEW LOGICS")
        #print(self.game_input)

        # Step 2: Initialize world
        #print("2")
        self.initialize_world()
        #self.save_game_snapshot()

        # Step 3: Add actions
        #print("3")

        look = ActionTemplate('look', 'Display {player.observation}', 'None')
        inventory = ActionTemplate('inventory', 'Display {inventory}')
        go = ActionTemplate('go to {room1}', 'Move {player} to {room1}; Display You have now entered {room1}; Display {player.observation}')
        #take = ActionTemplate('take {item1}', 'Move {item1} to {inventory}; Display You have now taken {item1}; Display {player.observation}')
        self.game.add_action_template(look)
        self.game.add_action_template(inventory)
        self.game.add_action_template(go)
        #self.game.add_action_template(take)

        MAXIMUM_ATTEMPT = 1
        success_count = 0.0
        count = 0.0
        length = 0.0
        failed_actions = []
        for sentence in self.game_input['game_logics']:
            length += 1
            #print("ADDING NEW SENTENCE FROM GAME LOGICS")
            #print(sentence)
            #print(self.game_input["game_logics"][sentence])
            previous_attempts: List[Tuple[str, str]] = []
            check = False
            for i in range(MAXIMUM_ATTEMPT):
                count += 1
                log('build_game', f'Adding sentence: {sentence}\tAttempt: {i+1}/{MAXIMUM_ATTEMPT}')
                is_successful, llm_raw_output, message, new_action_template = self._try_adding_action(sentence, previous_attempts=previous_attempts)
                check = check or is_successful
                if is_successful:
                    #print("SUCCESFUL ADDITION OF ACTION:")
                    #print(new_action_template)
                    success_count += 1
                    break
                else:
                    print("INVALID ATTEMPT")
                    previous_attempts.append((llm_raw_output, message))
            if not check:
                failed_actions.append(sentence)
            #    raise RuntimeError(f'Maximum number of attempts failed when adding action {sentence}!')
            #self.save_game_snapshot()

        #print("SUCCESSFUL PERCENTAGE OUT OF TRIES, SUCCESSFUL ACTIONS")
        #print(success_count / count, success_count / length)

        # Step 4: Restore game map to original state, and populate missing attributes
        #print("RESULT BEFORE RESTORATION")
        #print(self.game.world.serialize)

        room_count = 0
        item_count = 0
        char_count = 0

        items = []

        for node in self.game.world.nodes:
            if type(node) is Room:
                room_count += 1
                continue
            if type(node) is Item:
                item_count += 1
                items.append(node)
                continue
            if type(node) is Character:
                char_count += 1
                continue
            #print(node, node.container, node.original_container)
        
        #print("NODE COUNTS")
        #print(room_count, char_count, item_count)
        #print(items)

        self.game.world.restore_initial_state()
        self.game.happened_events = set()
        self.game.happened_actions = set()
        self.populate_node_attributes()

        print("FAILED ACTIONS", failed_actions)
        
        return self.game


game_builder = GameBuilder(game_input)
adventureGame = game_builder.build_game()
game_builder.game.world.save('game_builder_output.json')
result = game_builder.game.world.serialize()
print(result)

[Room]@(facility-perimeter-b5abea14-dc09-4576-8799-c363ed3a7008) <class 'nodes.Room'> (0, 0)
get_world: Adding Room facility perimeter to world
[Character]@(outer-guards-73704ceb-7466-4444-8aae-17bbacd7bf97) <class 'nodes.Character'> [Room]@(facility-perimeter-b5abea14-dc09-4576-8799-c363ed3a7008)
get_world: Adding Character outer guards to world
[Room]@(control-room-03aaf095-95cb-4c77-9af6-93a91dd614dd) <class 'nodes.Room'> (0, 1)
get_world: Adding Room control room to world
[Item]@(surveillance-terminal-378da4d9-6cd6-4148-a35d-d7f523b58af1) <class 'nodes.Item'> [Room]@(control-room-03aaf095-95cb-4c77-9af6-93a91dd614dd)
get_world: Adding Item surveillance terminal to world
[Room]@(server-room-69cf7fb2-e165-46fa-a857-a8ab7e607db9) <class 'nodes.Room'> (0, 2)
get_world: Adding Room server room to world
[Item]@(bomb-access-code-b308c810-21d3-4729-b340-22e44f8ac101) <class 'nodes.Item'> [Room]@(server-room-69cf7fb2-e165-46fa-a857-a8ab7e607db9)
get_world: Adding Item bomb access code to wo

In [12]:

print(adventureGame.commands)
print(adventureGame.action_history)
serial = adventureGame.world.serialize()
print(serial['rooms'])
print(serial['characters'].keys())
print(serial['items'].keys())


['sneak past {outer guards} at {facility perimeter}', 'hack into {surveillance terminal} at {control room}', 'extract {bomb access code} in {server room}', "neutralize {bomb technician} at {technician's quarters}", 'disarm {bomb} with {bomb access code} at {bomb site}']
[<result.ActionResult object at 0x169e7a390>, <result.ActionResult object at 0x169e8ef90>, <result.ActionResult object at 0x16832fcd0>]
{'facility-perimeter-b5abea14-dc09-4576-8799-c363ed3a7008': {'name': 'facility perimeter', 'description': 'You are at facility perimeter.', 'container': None, 'id': 'facility-perimeter-b5abea14-dc09-4576-8799-c363ed3a7008', 'items': ['set', []], 'characters': ['set', ['[Character]@(outer-guards-73704ceb-7466-4444-8aae-17bbacd7bf97)', '[Player]@(player-4013e790-0502-4bfe-89e1-26d22287b8d7)']], 'adjacent_rooms': {'east': '[Room]@(control-room-03aaf095-95cb-4c77-9af6-93a91dd614dd)', 'west': None, 'north': None, 'south': None, 'inside': None, 'outside': None}, '_type': 'Room'}, 'control-roo

In [13]:
from game_construct_prompt import expand_sentence, analyze_action, populate_attribute, get_verbs, generate_new_preconditions, check_if_existing_action, measure_coherence

In [14]:
def addNewAction(adventureGame, llm, user_input, currRoom):
    llm_raw_output, base_form, arguments, attribute_list, add_attribute_list, subject, preceding_events, effects, action_template = generate_new_preconditions(adventureGame, llm, user_input + "; room: " + currRoom, [])
    try:
        found = adventureGame.world.find_node(subject)
    except:
        print_warning("The subject of your action does not exist.")
        return
    valid_rooms = []
    for node in adventureGame.world.nodes:
        if type(node) is Room:
            valid_rooms.append(node)
    ###Adding new items randomly to the world
    #print("VALID ROOMS")
    #print(valid_rooms)
    for obj in arguments.values():
        try:
            found = adventureGame.world.find_node(obj)
        except:
            rand_room = np.random.choice(valid_rooms)
            new_obj = Item(obj, obj)
            adventureGame.world.add_node(new_obj, rand_room)
                        
    ###Registering new attributes with corresponding objects

    for attribute_name, belonging_class_name, attribute_type in attribute_list:
        #print("REGISTERING NEW ATTRIBUTE", attribute_name)
        #print(attribute_name, belonging_class_name, attribute_type)
        belonging_class = Node.mapping[belonging_class_name]
        #print(belonging_class, belonging_class.has_attribute(attribute_name))
        if not belonging_class.has_attribute(attribute_name):
            #print("ACTUALLY REGISTERING", attribute_name)
            belonging_class.register_new_attribute(
                attribute_name, attribute_type, None)
    
    for add_attribute, belonging_class_name, attribute_type in add_attribute_list:
        #print("CHECKING FUTURE EVENTS")
        attribute_input = subject + "." + add_attribute
        for action in adventureGame.actions.keys():
            if subject in action:
                is_necessary, new_attribute = check_future_events(llm, adventureGame.actions[action], attribute_input)
                #print(is_necessary, type(is_necessary), new_attribute, type(new_attribute))
                #print("OLDCONDITIONS")
                #print(adventureGame.actions[action].conditions)
                proc = adventureGame.actions[action].conditions.processed_condition_expression
                cond_field = adventureGame.actions[action].conditions.condition_fields
                #print(proc, cond_field)
                #print(proc, type(proc))
                numbers = re.findall(r'\d+', proc)  # Find all sequences of digits
                max_field_num = str(max(map(int, numbers)) + 1)
                new_field_index = " and FIELD_" + max_field_num
                #print(proc + " and FIELD_5")
                #print(cond_field, type(cond_field))
                newProc = proc + new_field_index
                newField = ConditionFieldFactory.create_condition_field(new_attribute)
                #print(newProc, type(newProc))
                #print(newField, type(newField))
                cond_field[new_field_index] = newField
                newConditions = ComplexCondition(newProc, cond_field)
                #print("NEWCONDITIONS")
                #print(newProc, cond_field)
                #print(newConditions)
                adventureGame.actions[action].conditions = newConditions
        
            
    #print("sub 4")
    #print(action_template)
    adventureGame.add_action_template(action_template)
    #print("BASEFORM")
    #print(base_form)
    command = replace_placeholders(base_form, arguments)
    #print(command)
    action = adventureGame.get_action(command)
    #print("ACTION")
    #print(action, action.conditions, action.operations)
    for flag in preceding_events:
        action.flags.append(flag)
        #print("ADDING NEW PRECEDING EVENT")
        addNewAction(adventureGame, llm, flag + "; preceding: false.", currRoom)
    assert action is not None, "Bug! Action is none!"
    #print("GETTING FIXES")
    fixes = action.conditions.get_fixes_complex(adventureGame, llm=llm)
    #print("FIXES:")
    #print(fixes)
    for fix in fixes:
        fix.apply(adventureGame.world)
    
    #print("EXECUTING ACTIONS")
    #print(action)
    action_result = action.execute(adventureGame)
    adventureGame.happened_events.add(user_input)
    #print(action_result)
    #print("sub 6")

    # STEP 6: Check if the action succeeds.
    #print(adventureGame.action_history)
    action_result = adventureGame.action_history[-1]
    #if not action_result.success:
        # REMOVE THE ACTION IF ALREADY ADDED
        #print("REMOVE NOT WORKING ACTION", adventureGame.remove_action(action_template))
    
    #print("sub 7")
    effects = effects.split(';')

    score = measure_coherence(llm, user_input, llm_raw_output, ''.join(preceding_events))
    #print("SCORE")
    #print(score)

    # STEP 7: Check if the action has the correct effects.
    #desired_effects = [f'{{{effect}}}' for effect in effects]
    #actual_effects_expression = ' and '.join(desired_effects)
    #print(actual_effects_expression)
    #actual_effects_condition = ComplexCondition.build_from_string(
    #    game = adventureGame, expression = actual_effects_expression)
    #is_satisfied, messages = actual_effects_condition.evaluate(
    #    adventureGame)
    #print("ACTUAL EFFECTS SATISFACTION CHECK")
    #print(is_satisfied, messages)
    adventureGame.commands.append(command)
    adventureGame.world.restore_initial_state()
    adventureGame.happened_events = set()
    adventureGame.happened_actions = set()
    game_builder.populate_node_attributes()

In [15]:
from type import GameState
import numpy as np
from condition import ConditionFieldFactory
adventureGame.execute_command('look')
while True:
    user_input = input('Enter a command: ')
    if user_input == "exit":
        break
    print("CHECKING EXISTING ACTION")
    print(user_input)
    print(list(adventureGame.actions.keys()))
    isMatch, user_input = check_if_existing_action(llm, user_input, list(adventureGame.actions.keys()))
    newAction = adventureGame.execute_command(user_input)
    #print("CURR ROOM")
    currRoom = adventureGame.world.player.container.name
    if newAction:
        addNewAction(adventureGame, llm, user_input + "; preceding: either; ", currRoom)
    if adventureGame.game_state != GameState.UNFINISHED:
        print_warning('You win!' if adventureGame.game_state == GameState.WON else 'You lose!')
        break


[Room]@(facility-perimeter-b5abea14-dc09-4576-8799-c363ed3a7008)
True {}
Room: facility perimeter
Description: You are at facility perimeter.
No items in the room.
Characters in the room:
  - outer guards: An outer guards
Adjacent rooms:
  - East: control room
Your inventory is empty.
Your goal: 

Room: facility perimeter
Description: You are at facility perimeter.
No items in the room.
Characters in the room:
  - outer guards: An outer guards
Adjacent rooms:
  - East: control room
Your inventory is empty.
Your goal: 

Room: facility perimeter
Description: You are at facility perimeter.
No items in the room.
Characters in the room:
  - outer guards: An outer guards
Adjacent rooms:
  - East: control room
Your inventory is empty.
Your goal:
--------------------------------------------------


In [ ]:
# Sample agent to play through games ideally
from type import GameState

class GameAgent:
    def __init__(self, game, game_logics):
        """Initializes agent with game instance and game logistics

        Args:
            game_logics (dict): The game logics containing ordered actions
        """

        self.game = game
        self.game_logics = game_logics['game_logics']
        self.actions = self.game_logics.keys()

    def play(self):
        '''
        Plays through the game by executing each action in game_logics sequentially
        '''
        
        for i in self.actions:
            action = i.replace("adventurer ", "")
            action = action + " " + self.game_logics[i]["location"][0]
            action = re.sub(r'[^\w\s]','', action)
            #print("ACTION", action)
            self.game.execute_command(action)
            
            if self.game.game_state != GameState.UNFINISHED:
                print("Game Over")
                if self.game.game_state == GameState.WON:
                    print("You Win")
                else:
                    print("You lose")
                break

        print("\nGame Agent has finished executing all actions.")

# Restore to original state before playing
# adventureGame.world.restore_initial_state()
# adventureGame.world.move_node(adventureGame.world.player, adventureGame.world.find_node("forest"))

myAgent = GameAgent(adventureGame, game_input)
myAgent.play()


# Restore to original state after playing
adventureGame.world.restore_initial_state()
print(myAgent.actions)